# VCC Submission Notebook

Hello! 

This is a notebook that will help you prepare your predicted AnnData to be ready to be scored by `cell-eval` against a validation dataset.

Before we begin you will need a few things:

1. `cell-eval` installed and in your `$PATH` (see our [installation guide](https://github.com/ArcInstitute/cell-eval?tab=readme-ov-file#installation))
2. The number of expected cells / perturbation in the validation dataset (CSV) ([download](https://virtualcellchallenge.org/app))
3. The gene names to predict (CSV) ([download](https://virtualcellchallenge.org/app))
4. Your model predictions in an AnnData (h5ad)
5. (Optional) The training AnnData (if you are not predicting Non-Targeting Controls) ([download](https://virtualcellchallenge.org/app))


> Note: Your model predictions **may not exceed 100K cells total**

## Building an Example Submission

For the purposes of this tutorial we will be generating **random predictions** and preparing them to be evaluated.

We will create an AnnData with *random gene abundances* for each cell, where the number of cells for each perturbation matches the number of cells in the validation dataset.

### Load in our Expected Counts

In [ ]:
import polars as pl

# Define our path
pert_counts_path = "./pert_counts_Validation.csv"

# Read in the csv
pert_counts = pl.read_csv(pert_counts_path)

# Show the dimensions
print(f"Dimensions: {pert_counts.shape}")
pert_counts.head()

In [2]:
import polars as pl
# Define our path
pert_counts_path = "./test_gene_ncell_1119.csv"
# Read in the csv
pert_counts = pl.read_csv(pert_counts_path)
# Show the dimensions
print(f"Dimensions: {pert_counts.shape}")
pert_counts.head()

Dimensions: (30, 2)


target_gene,n_cells
str,i64
"""MED13""",2787
"""STAT1""",2493
"""KAT2A""",2120
"""KDR""",2056
"""HMGN1""",2004


### Load in our Expected Gene Names

In [3]:
gene_names_path = "./gene_names.csv"

# Read this in and immediately convert to array
gene_names = pl.read_csv(gene_names_path, has_header=False).to_numpy().flatten()

gene_names

array(['SAMD11', 'NOC2L', 'KLHL17', ..., 'MT-ND5', 'MT-ND6', 'MT-CYB'],
      dtype=object)

### Define our random predictor

In [4]:
import numpy as np
import pandas as pd
from numpy.typing import NDArray
import anndata as ad
from typing import Union


def random_predictor(
    pert_names: NDArray[np.str_],
    cell_counts: NDArray[np.int64],
    gene_names: NDArray[np.str_],
    max_count: Union[int, float] = 1e4,
    log1p: bool = True,
) -> ad.AnnData:
    """Generate a random AnnData with the expected number of cells / perturbation.
    This is a dummy function that is meant to stand-in for a perturbation model.
    """
    matrix = np.random.randint(
        0,
        int(max_count),
        size=(
            cell_counts.sum(),
            gene_names.size,
        ),
    )
    if log1p:
        matrix = np.log1p(matrix)
    return ad.AnnData(
        X=matrix,
        obs=pd.DataFrame(
            {
                "target_gene": np.repeat(pert_names, cell_counts),
            },
            index=np.arange(cell_counts.sum()).astype(str),
        ),
        var=pd.DataFrame(index=gene_names),
    )

### Run our random predictor

In [5]:
adata = random_predictor(
    pert_names=pert_counts["target_gene"].to_numpy(),
    cell_counts=pert_counts["n_cells"].to_numpy(),
    gene_names=gene_names,
)
adata

AnnData object with n_obs × n_vars = 39945 × 18080
    obs: 'target_gene'

### Adding in Non-Targeting Controls if you are not predicting them

Our evaluation framework expects non-targeting controls to be included in the predicted AnnData, but not all models may try to predict non-targeting controls.
If you are not predicting non-targeting controls, you can take the non-targeting from the training AnnData and just copy them over into your predicted AnnData for validation.

In [ ]:
# Define our path to the training anndata
tr_adata_path = "./adata_Training.h5ad"

# Read in the anndata
tr_adata = ad.read_h5ad(tr_adata_path)

# Filter for non-targeting
ntc_adata = tr_adata[tr_adata.obs["target_gene"] == "non-targeting"]

# Append the non-targeting controls to the example anndata if they're missing
if "non-targeting" not in adata.obs["target_gene"].unique():
    assert np.all(adata.var_names.values == ntc_adata.var_names.values), (
        "Gene-Names are out of order or unequal"
    )
    adata = ad.concat(
        [
            adata,
            ntc_adata,
        ]
    )

### Write our predictions to some output path

In [7]:
adata.write_h5ad("./eval/example.h5ad")

## Running `cell-eval prep`

Now that we have our predictions, we will run `cell-eval` to prepare our AnnData for competition scoring.

```bash
cell-eval prep \
    -i ./example.h5ad \
    --genes ./gene_names.csv
```

And that's it! Your model outputs will be output to path: `./example.prep.vcc` are ready for scoring.